# Unsupervised Clustering of Microscopy Images
----
Last Update: 25 Oct 2016

## Acknowledgements
- Juan Nunez-Iglesias (Supervisor) - for continuous support and advice throughout the project.
- Adrian Hecker - for seeting the stage for this project, and laying the groundwork with feature-based encoding.

## Contents - Chapter 1
1. High Content Screening
2. About the Data
3. Machine Learning
4. Sparse Representation & Dictionary Learning
5. Bag-of-Words Modelling - TF-IDF

----

# Chapter 1 - Introduction

## 1. High Content Screening
*High-content screening* refers to a suite of technological systems, methods and applications related to medical imaging.  The ‘*high*’ in high-content refers to the large volumes of data output.  High-content screens have been made possible by the advent of *RNA interference* (RNAi) technology, which allows researchers to selectively suppress of some targeted gene of interest.  For instance, in studies that aim to discover the function of some gene of interest, researchers some can isolate that gene, disable it using RNAi, and take a microscopy photograph of those cells to try and observe any subsequent cellular effects. They can compare samples where the targeted gene has been suppressed with samples where the gene is allowed to function normally, and see if there's any difference between the two groups of samples.

Researchers may prepare a predetermined set of descriptors – or *features*, in the parlance of computer science – such as cell number, roundness, size, density, or number of nuclei.  Each feature may be mathematically represented as a single dimension.  A large number of these features would equivalently yield high-dimensional representations of the data, or a *high-dimensional feature vector*. Current HCS algorithms are able to profile cellular information using these feature vectors.  The aptly-named CellProfiler software, for instance, allows users with minimal training in image analysis to characterize cell images in terms of a number of quantitative traits. 

----
![eggs](https://raw.githubusercontent.com/Don86/mic-scrapbook/master/eggs.jpg)
*Profiling microscopy images of cells in terms of biological features*

#### Drug Discovery - Targeted and Phenotypic Screens
*Targeted screens* are studies centred around specific biological hypotheses; for example: “Are cancerous cells less round than normal cells?” ([Yes](https://en.wikipedia.org/wiki/Dysplasia#Microscopic_changes)).  These are based on a pre-specified unit of study, such as a gene, phenotype, or compound of interest.  Targeted screens allow the use of some biological knowledge to confirm or refute the hypothesis.  

*Phenotypic screens* are exploratory studies which approach the data without any prior hypothesis.  The objective of a phenotypic screen is not to investigate in depth any single hypothesis, but to assess the data for what hypotheses are possible.  Phenotypic screens are useful because less prior chemical or biological knowledge of the domain is required, and casting a wide net without prior expectations is more unbiased than a targeted search of some phenotype of interest.  In other words, researchers aren't looking for an answer to some specific question, they’re looking for what questions to ask.  As such, phenotypic screens are handy as a first pass to identify potential promising areas of study for further analysis. 

#### Computational Challenges in HCS
The techniques and methods for quickly generating and profiling large volumes of images has rather outpaced the rate of development of bioinformatics methods to analyse all that data.  Manually processing the large volumes of output data is laborious and time-consuming, often requiring expert human input. This is compounded by the fact that high-content screening studies are likely to be exploratory studies, so that in order to reap the most information out of the data, researchers are often forced to use a large number of features, in the order of hundreds or thousands, in the spirit of “*measure everything and ask questions later*”.  

Using many features at once can be computationally overwhelming, requiring entirely new suites of methods to do analysis with such high dimensions.  For example, statistical tests for multiple comparisons – testing hundreds and thousands of hypotheses of the same data set – present a new level of complexity not previously encountered with using only a few features, such as the very large number of false positives that can arise, or dealing with correlation between the hypotheses. Using so many features, some of which may be excessive, may also subsequently present a feature selection problem: out of all these features, which are the most “useful” in some sense?  The operative meaning of “useful” depends on context. For example, we may wish to prune out the most useless features in order to reduce computational load, while not sacrificing descriptive power.  Or, if we decide to target a particular phenotype of interest is for study, we may wish to select only a few features that are most descriptive of that phenotype. Discovering which features most relevant to the phenotype is also of scientific interest, such that these can be subsequently used for prediction. 

As such, even though microscopy images can contain a wealth of information, targeted studies  still tend to use a minimal number of features to reduce mathematical complexity, trading off exploratory power for a more tightly targeted investigation.  Nevertheless, even when not being utilized at its maximal capabilities, HCS is still a mainstay of the pharmaceutical industry, and has been demonstrably effective in clinical applications such as drug screening, or for verifying the molecular-mechanism-of-action. 

## 2. About the Data
I used the image set [BBBC021v1](https://data.broadinstitute.org/bbbc/BBBC021/) [Caie et al., Molecular Cancer Therapeutics, 2010], available from the [Broad Bioimage Benchmark Collection](http://www.nature.com/nmeth/journal/v9/n7/full/nmeth.2083.html). This is a set of microscopy images of breast cancer cells, treated with a compendium of 113 drug treatments. Each image has the dimensions 2048 by 2560 pixels in height and width, and 3 colour channels: red, green and blue (R, G, B).  BBBC021 comprises 13,200 image files, each a treatment sample of some compound, the concentration of that compound, and, if known, a method-of-action (MOA) of that particular treatment.  Different compounds can have the same MOA, but no compound can have multiple MOAs.  Different levels of concentration of the same compound will all have the same MOA. There are 103 MOA labels in all, comprising 38 compounds at 1-7 levels of concentrations each. 

However, not all images have had their MOA identified. In all, 962 images were completely annotated with compound, concentration and MOA labels.  The following table presents some characteristics of the compound-concentration to MOA proportions, and relative proportions of each class according to different levels of class labels. 

The original study that BBBC021 was generated for was interested in developing an automated method of identifying MOAs of a given chemical treatment.  It was motivated by the fact that high-content screens have been successfully used to draw correlations between certain chemical compounds and certain clinical reactions, but knowledge of exactly how drug X induces physiological reaction Y is still largely unknown.  Given that biological pathways (sequences of chemical reactions in the body) can be long and highly intertwined affairs, researchers may not even know where to start looking.  

The study then proceeded to characterize microscopy images in terms of 150 features, such as cell shape, nuclear area, nuclear intensity, and so on.  This feature-representation of each image made the data amenable to further analysis, for instance to tongue-twisting questions like “*what is the effect of increasing concentrations of the microtubule-disrupting drug colchicine on the transient mitotic arrest phenotype?*”. 

My project is the direct successor of a similar investigation by Adrian Hecker.  Adrian proceeded in rather the same fashion as the DMSO study, but this time eschewing many of the domain-specific features, instead using more generic image-oriented features, such as “*how blue this image is*”.  Blue chemical dye actually does have biological significance – it’s a blue-fluorescent chemical 4’, 6-diamidino-2-phenylindole (DAPI), particularly effective at staining DNA, thus overall useful for staining nuclei – but our machine doesn’t need to know that.  He found that the 246 features in the `Microscopium` library were quite sufficient to classify the different MOAs, with about 85% classification accuracy.  Rather more surprisingly, MOA classification performance remained high even when random subsets of the 246 features were used, making the rather paradoxical implication that randomly chosen subsets of features are quite useful, but a combination of all features does not markedly increase classification performance. 

Given that image profiling based on handcrafted features has already be done quite exhaustively, my project focuses on image profiling without handcrafted features.  The immediate problem is then to discover what the automatically-generated features might be. To this end, we turn to dictionary learning methods, in the field of machine learning. 


## 3. Machine Learning
*Machine learning* is a broad field of computational methods which allows computers to learn, in the sense of autonomously discovering new information from a given set of data, without having to be explicitly programmed. A caricature of (supervised) machine learning is as follows:
1.	From a large set of pictures of animals, have our machine make an intelligent guess as to whether a randomly selected picture is that of a cat.  The machine will likely make a purely random guess at first.
2.	Do step 1 repeatedly, “rewarding” or “punishing” the machine for every correct or wrong answer. 

Over a sufficiently large number of iterations, the machine will eventually consistently make correct guesses as to whether pictures presented to it are pictures of cats, or not.  It can do so despite the fact that it does not actually know what a ‘cat’ actually is, or, for that matter, what an ‘animal’ or a ‘picture’ is, or who is the ‘self’ who is making all these guesses (though for the sake of preserving humanity’s status as dominant species on this planet, perhaps it’s better that we not conduct too much research on machine sentience).  Whether or not it is meaningful to say that the machine knows or does not know what a cat is, given that it has developed a very accurate description of a cat, albeit one that’s only intelligible to itself (machine-readable), is a [philosophical argument](https://en.wikipedia.org/wiki/Knowledge_argument) beyond the scope of this paper.

In the broad field of machine learning, the subfield of computer vision is concerned with the class of machine learning methods centred around pictures.  Some such examples are <i>handwritten-digit recognition</i> and <i>facial recogition</i>, developed on more unstructured, but still fairly idealized datasets like MNIST or CIFAR100.  We say that these datasets are structured because we can quite easily point out discrete, human recognizable elements in them, and the object of interest is usually placed in the centre of the image; the only difficulty is in doing so for tens of thousands of images.  The purpose of this project is to repurpose machine learning methods for use on the messier, more unstructured microscopy image data. 

We have a few practical considerations to guide our design choices:
- *No handcrafted features* - The utility of the existing set of 246 features in `Microscopium` has already been explored by previous work; we are now interested in how well we can characterize the microscopy images without handcrafted features.  This is of immediate practical use since this mitigates the need for expert input to craft useful features, and is of computational interest since the features themselves are inferred directly from the data, rather than using generic feature descriptions.  
- *No class labels* – previous work relied on training neural networks and classifiers from labelled training data. However, we would like to advance to a step where we need not use class labels at all.
- *Minimal data preprocessing* – other than for images that obviously contain technical errors, we would like to minimize any necessary data preprocessing that makes the data more amenable to any particular method, such that the methods developed can be deployed in as large a class of problems as possible.  For instance, image segmentation could be used for object identification, by highlighting edges more clearly and clearly demarcating foreground and background, but this presumes an image which does contain depth. 

This seems like an impossible trinity; as reflected by the '*[no free lunch](https://en.wikipedia.org/wiki/No_free_lunch_in_search_and_optimization)*' theorems – in terms of an objective function, any increase(decrease) in discriminative ability would necessarily require a decrease(increase) in reconstructive or predictive ability, such that any two objective functions would be equivalent if their performance was averaged across all possible problems. 



## 4. Sparse Representation & Dictionary Learning
We start out by defining some terms.
- *Sparse representation* – this does not refer to any one methodology, but is instead a general design principle of encoding an image based on only its most important visual attributes.  Generally speaking, we assume that a natural image consists of “important” and “unimportant” elements.  A good sparse representation should be a simpler representation of the image, yet remain sufficiently descriptive of the image to minimize reconstruction error when the image is rebuilt from its sparse encoding.  What’s important or not is mechanically decided by the sparse-encoding algorithm; for purposes of human understanding, we can think of it in terms of an artist’s sketch versus a fully-drawn painting – the sketch provides the outline, the rest is just filling in the details. 

- *Dictionary* – Now we imagine that we can somehow parse an image into a bunch of discrete visual elements – for instance, we might imagine that a picture of a cat may consists of visual elements like paws, whiskers, a patch of coat colour, etc.  The visual elements of many cats would then contain different physiological features of different cats, such as eye colour, different coat colours, tail lengths, etc.  A *dictionary* then consists of all these visual elements, rather like a checklist, such that if we want to describe one particular cat, we can look up this dictionary and select the most suitable visual elements that most aptly describe our cat.  This is rather the same way that the [English dictionary](http://www.dictionary.com/browse/dictionary?s=t)[1] contains all possible words, from which we choose the correct words to construct a sentence with.

- *Dictionary learning* – A dictionary may be constructed either using handcrafted features, e.g. using our knowledge of what physiological features cats may have, or without – the latter methodology of fitting the most suitable dictionary based on the input data is known as dictionary learning.

- *Sparse encoding* – a spare encoding algorithm computes a sparse representation of a particular image (cat), based on a dictionary of visual elements (cat pictures). While the two are quite synonymous in the literature, for the sake of simplcity, we henceforth use ‘encoding’ as a verb (e.g. we sparse-encode the image of a cat), and ‘representation’ as a noun (e.g. ‘we found that the sparse representation of our cat selected the orange coat colour’). 

[1] *I couldn't help but demonstrate the circular logic of consulting a dictionary on the meaning of the word 'dictionary'.*

We now elaborate on each of the components of the sparse representation procedure in turn:
### Sparse-Representation
Sparse representation has been used quite successfully in various audio, image and video processing domains, such as in image classification, image [enhancement](http://tvtropes.org/pmwiki/pmwiki.php/Main/EnhanceButton) or [*super-resolution*](http://xujuefei.com/felix_icip15_sr.pdf), and denoising applications.  This section reviews a number of papers which highlight key concepts, an introduces a denoising example to demonstrate the sparse representation procedure. 

### Constructing the dictionary
A dictionary is a set of a very large number of possible descriptors of the data, usually represented by a 2D matrix <b>D</b>, where the columns are the numerical representations of those descriptors.  The columns of a dictionary are canonically known as *atoms*.  If there is sufficient knowledge of the domain, we can use handcrafted features to form the dictionary. Otherwise, we use a dictionary learning algorithm to directly discover from the data what some possible descriptors of the data might be. The data can be a single image, or a set of multiple images, which may or may not have class labels; all depending on the nature of the study. 

There exists an intermediary between using a set of pre-determined features, versus using none at all, which is to use a set of mathematical functions of the data – this way, we are still using statistical or geometric aspects derived wholly from the data, though the function which computes these aspects is pre-determined.  For instance, *Eigenfaces* <citation> does facial recognition using eigenvectors. Notably, a number of papers <citation> have noted that in sparse representation applications without handcrafted features, the choice of features is not critical, as long as there is a large enough number of features.  

In the context of multiple image studies, sparse representations have also been successfully been used in classifying and recognizing faces from different individuals, and classification of texture images.  In particular, [Huang and Aviyente](https://papers.nips.cc/paper/3130-sparse-representation-for-signal-classification.pdf) distinguish between *discriminative* and *reconstructive* methods in the class of sparse encoding models. Discriminative methods, such as linear discriminant analysis (LDA) explicitly deal with encodings which distinguish between classes, but are less adept at signal reconstruction with minimal error.  These methods will incorporate concepts of variability within and between classes, class separation, and outlier detection.  On the other hand, reconstructive methods, like the K-SVD algorithm we'll be using, primarily aim to reconstruct an input signal as accurately as possible. These do not consider discrimination between classes at all. 

### K-SVD
*<All maths; omitted.*>

### Sparse Encoding
Now, using our dictionary <b>D</b>, and given an input vector <b>y</b>, we would like to represent <b>y</b> as a linear combination of the fewest possible columns of <b>D</b> as possible. <b>D</b> is said to be *overcomplete* if it has many more columns than rows; mathematically, this yields an overcomplete linear system with an infinite number of solutions, such that we can find a sparse solution. Intuitively: on the social media platform Twitter, using only a few hashtags, out of tens of thousands of possible hashtags, is quite sufficient to describe the topic of a post.  This works precisely because there are tens of thousands of possible hashtags, so using only a few is sufficient to home in on a particular topic. 

We have a few choices of sparse encoding algorithms, where we focus on *LASSO LARS*, and *orthogonal matching pursuit*. These are two ways of doing the same thing, each with their advantages and disadvantages.

----

<b>Example 1 (Denoising)</b> – Given a single image, corrupted with white noise, our objective is to recover the original image as best as we can. See the reference code structure on sklearn's example [here](http://scikit-learn.org/stable/auto_examples/decomposition/plot_image_denoising.html).

![gel](https://raw.githubusercontent.com/Don86/mic-scrapbook/master/gellybean1.png)

Note that the original image shown on the left is only for our reference; the dictionary learning algorithm never sees the original, only the noisy image. Also note that in this example, there is a clear distinction between the semantics of ‘signal’ and ‘noise’ – a luxury which we will not always have.

We’ll produce multiple samples from the noisy image by extracting patches from it, where each patch is a 2D matrix with dimensions (10 pixels, 10 pixels).  The exact patch extraction procedure will be described later; for now, it suffices to know that, like a jigsaw puzzle, we are able to fragment and reassemble a picture using its constituent patches, using the inbuilt patch extraction and reconstruction methods in Python’s `sklearn` library.  We’re able to extract about 300,000 from the image. 

We'll learn a dictionary using the `MiniBatchDictionaryLearning` method in `sklearn`, using orthogonal matching pursuit as the sparse-encoding algorithm, arbitrarily setting the number of columns to 200 and regularization parameter  = 1. We can visualize each of the columns of the dictionary as images in their own right; the following figure shows the first 100 columns:

![gel](https://raw.githubusercontent.com/Don86/mic-scrapbook/master/gellybean_V.png)

Unfortunately, it’s not easy to make sense of the visualized columns in terms of how they relate to the original image. It could be that the patch size is too small; (10, 10) isn't going to show a lot of information. Another [dictionary learning example](http://scikit-learn.org/stable/auto_examples/cluster/plot_dict_face_patches.html) using the Olivetti faces dataset has rather more interpretable (and haunting) dictionary components. Nevertheless, we now carry out sparse encoding, setting the number of nonzero coefficients to 1 – that is, each patch is only allowed to select one column in the dictionary that best describes it. 

Finally, we reconstruct the original image by taking the matrix product of our sparse representation matrix <b>X</b>, and the learnt dictionary <b>D</b>.

![gel](https://raw.githubusercontent.com/Don86/mic-scrapbook/master/gellybean_recon.png)

It's a pretty good reconstruction, removing grain from the corrupted image, and recovering distinct lines and shapes.

----

## 5. Bag-of-Words Representation: TF-IDF
In conventional *natural language processing* and text mining, the bag-of-words model is used to represent a document in terms of only a few key words (thus ‘bag-of-words’).  The bag-of-words representation of a collection of documents can then be classified according to classes (or ‘topics’), according to the frequency of occurrence of topic-specific keywords in that particular document. 

### "Bag-of-Atoms" in Computer Vision
In this section, we define a patch as a small rectangular selection of an image, which can be 3-dimensional – incorporate the three RGB colour channels – or 2-dimensional, meaning that the patch has been gray-scaled. 

The equivalent of a document in computer vision is an image, and the ‘words’ are descriptive patches of some sort.  Images are similarly represented in terms of a set of visual elements, where these visual elements can be pre-determined, or discovered from the image directly, as we've done with dictionary learning.  The process of discovering what constitutes an “interesting” visual element is commonly called feature detection.  These visual elements can sometimes (but not necessarily!) be confirmed to be human-recognizable elements of the original image that are indeed distinct to that image, e.g. whiskers on a cat. 

One such method of discovering such visual elements is scale-invariant-feature-transform (SIFT).  In its standard usage, SIFT is given a training set of some object of interest (such as a cat) in many different orientations and scales, from which it learns the distinctive features of that object – in particular, it looks for features which are invariant to scaling, translation and rotation; this invariance is what constitutes “distinctive”.  Given a test set of images comprising many objects, it will identify whether or not the object is in the test image, and its location in the image. 

A previous study made use of a *codebook* (the equivalent of a dictionary), where, given a set of visual features, ‘code words’ are computed as some kind of aggregation of several similar-looking visual features.  For example, we can use k-means to perform this aggregation, and set the code words to be the centres of the fitted clusters.  Each patch in an image can then be represented in terms of its membership to a cluster, such that the image can be, on the whole, represented in terms of a distribution of membership to different clusters. 

In our case, the dictionary learnt by K-SVD will serve as the codebook, a happy coincidence since K-SVD is regarded as a generalization of k-means <citation>. The sparse representation of each patch is equivalently the bag-of-words(atoms) representation according to the dictionary.  We can then aggregate the patches back into the images from which they originated from

### TF-IDF
In the original context of text mining, a *term frequency-inverse document frequency* (TF-IDF) vector is a vector representation of a document in terms of its keywords (or keyword vector), comprising of real or integer numbers. The *i*-th entry in the the TF-IDF vector  corresponds to the relative importance of *i*-th keyword, where the value of T(i) is a measure of that importance.  We denote the following: our collection of documents as C, where each document is indexed as *d*, *d = 1…nC*, where *nC* is the number of documents in the collection.  TF-IDF is then the product of two terms: *term frequency, tf*, and *inversed-document frequency, IDF*.

For a gist of ideas, we present the components of the TF-IDF identity: 
- <b>Term frequency</b> – This is a reflection of the importance of that term to the document, relative to all the other terms. If a term like ‘whiskers’, ‘feline’, ‘meows’ occur very frequently in some document of interest, and terms like ‘rain’ and ‘serendipitous’ occur very rarely, then that document is probably about cats. We compute the term frequency of a particular term *t* directly as the number of times that *t* occurs in document *d*.  
- <b>Inverse document frequency</b> – The idea behind this is that a term which occurs very frequently throughout all documents is less useful (such as ‘the’, ‘this’ – extremely ubiquitous grammatical terms known as *stop words*), so we wish to reduce their importance. 

Now that we've set the stage for the background and motivation of this project, we can go on to the specifics of implementing the entire procedure from chapter 2. 